In [4]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [5]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')

# AREA OF INTEREST
region_L1 = ee.Geometry.Polygon([[[-30, -40], [65, -30], [65, 40], [-30, 40]]])
region_json= [[-30, -40], [65, -30], [65, 40], [-30, 40]]

subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia'
}

In [6]:
subregion = 'ws'
year = 2010
name_asset = 'T_Annual'
selected_subregion = subregions[subregion]
filtered = countries_gaul.filter(ee.Filter.eq('subregion', selected_subregion))
country_number = filtered.size().getInfo()
print(country_number)
#for indice in range(0,country_number):
#    print(indice,filtered.getInfo()['features'][indice]['properties']['name'])
radice = 'projects/fao-wapor/'
raster_name = radice + name_asset + "/" + name_asset + "-" + str(year)
print(raster_name)
prefix = raster_name.split("/")[2]
raster_stats = ee.Image(raster_name).toFloat()
scale_calc = ee.Image(raster_stats).projection().nominalScale().getInfo()
print('Image scale: ', scale_calc)

14
projects/fao-wapor/T_Annual/T_Annual-2010
('Image scale: ', 248.48100621908284)


In [7]:
means = raster_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = scale_calc,
)

reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.sum(),
    sharedInputs = True
);

minMaxStds = raster_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = scale_calc,
);

In [11]:
features_m = means.getInfo()['features']
features_mms = minMaxStds.getInfo()['features']
df_m = pd.DataFrame(data=features_m[0:], columns = features_m[0])
df_mms = pd.DataFrame(data=features_mms[0:], columns = features_mms[0])
serie_m = df_m['properties'].apply(pd.Series)
serie_mms = df_mms['properties'].apply(pd.Series)
df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])
df_mms = pd.DataFrame(serie_mms[['fid', 'min', 'max','sum']])
df_stats = df_m.set_index('fid').join(df_mms.set_index('fid'), lsuffix='_df_m', rsuffix='_df_mms')
df_stats.index = df_stats.index.astype(int)
df_stats['year'] = year
df_stats['gee_asset'] = name_asset
for col in ['name', 'region', 'subregion', 'gee_asset']:           
    df_stats[col] = df_stats[col].astype('str')
df_stats

,gaul_code,iso3,mean,name,region,subregion,min,max,sum,year,gee_asset
fid,,,,,,,,,,,
135,137.0,KWT,5.810755,Kuwait,Asia,Western Asia,0.000,2283.159912,1.884480e+06,2010,T_Annual
139,141.0,LBN,360.913285,Lebanon,Asia,Western Asia,0.000,1935.243042,7.158174e+07,2010,T_Annual
278,-1.0,PSE,198.243447,Occupied Palestinian Territory,Asia,Western Asia,0.000,1676.406982,2.331422e+07,2010,T_Annual
279,-2.0,,416.870462,No Man's Land,Asia,Western Asia,13.325,1031.004028,6.390624e+05,2010,T_Annual
21,21.0,BHR,3.553262,Bahrain,Asia,Western Asia,0.000,2071.059082,4.290208e+04,2010,T_Annual
184,187.0,OMN,10.987296,Oman,Asia,Western Asia,0.000,3973.644043,5.894325e+07,2010,T_Annual
198,201.0,QAT,5.196946,Qatar,Asia,Western Asia,0.000,3863.151123,1.078605e+06,2010,T_Annual
212,215.0,SAU,12.145432,Saudi Arabia,Asia,Western Asia,0.000,5165.819824,4.176086e+08,2010,T_Annual
233,238.0,SYR,82.496171,Syrian Arab Republic,Asia,Western Asia,0.000,2406.603027,3.076891e+08,2010,T_Annual


In [144]:
file_name = name_asset + "_" + str(year) + "_" + subregion # + prefix.lower()
file_name
df_stats.to_csv('stats/' + file_name + '.csv')